<a href="https://colab.research.google.com/github/giangdip2410/Learn-NLP/blob/master/SentimentAnalysis_CNN2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!unzip "/content/imdb-dataset-of-50k-movie-reviews.zip" 

Archive:  /content/imdb-dataset-of-50k-movie-reviews.zip
  inflating: IMDB Dataset.csv        


In [3]:
import pandas as pd
data_pd = pd.read_csv("IMDB Dataset.csv")

data_pd["label"] = data_pd["sentiment"].map({'positive': 1, 'negative': 0})
data_pd.head()

,review,sentiment,label
0,One of the other reviewers has mentioned that ...,positive,1
1,A wonderful little production. <br /><br />The...,positive,1
2,I thought this was a wonderful way to spend ti...,positive,1
3,Basically there's a family where a little boy ...,negative,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,1


In [4]:
from keras.preprocessing.text import Tokenizer
t = Tokenizer(num_words=5000)
# fit the tokenizer on the documents
t.fit_on_texts(data_pd["review"].values)


Using TensorFlow backend.


In [5]:
len(t.word_index)

124252

In [0]:
X = t.texts_to_sequences(data_pd["review"].values)
y = data_pd["label"].values

In [7]:
X[0]

[27,
 4,
 1,
 79,
 2102,
 45,
 1072,
 12,
 100,
 147,
 39,
 307,
 3184,
 398,
 474,
 26,
 3195,
 33,
 23,
 203,
 14,
 11,
 6,
 621,
 48,
 596,
 16,
 68,
 7,
 7,
 1,
 86,
 148,
 12,
 3241,
 68,
 42,
 3184,
 13,
 92,
 2,
 134,
 4,
 570,
 60,
 268,
 8,
 203,
 36,
 1,
 661,
 139,
 1740,
 68,
 11,
 6,
 21,
 3,
 119,
 15,
 1,
 2333,
 38,
 11,
 119,
 2595,
 54,
 16,
 5,
 1479,
 376,
 38,
 570,
 92,
 6,
 3804,
 8,
 1,
 360,
 356,
 4,
 1,
 661,
 7,
 7,
 9,
 6,
 433,
 3184,
 14,
 12,
 6,
 1,
 358,
 5,
 1,
 2538,
 1064,
 9,
 2711,
 1421,
 20,
 538,
 32,
 4636,
 2468,
 4,
 1,
 1208,
 117,
 29,
 1,
 25,
 2970,
 2,
 391,
 34,
 6,
 21,
 299,
 20,
 1,
 4910,
 538,
 6,
 344,
 5,
 106,
 2453,
 2,
 51,
 34,
 327,
 2,
 23,
 110,
 225,
 243,
 7,
 7,
 10,
 58,
 131,
 1,
 280,
 1324,
 4,
 1,
 119,
 6,
 693,
 5,
 1,
 192,
 12,
 9,
 269,
 117,
 79,
 276,
 589,
 3024,
 834,
 180,
 1320,
 4161,
 15,
 2523,
 1243,
 834,
 1443,
 834,
 887,
 3184,
 149,
 954,
 183,
 1,
 86,
 398,
 10,
 123,
 210,
 3241,
 68,
 14,
 

In [0]:
from keras.preprocessing.sequence import pad_sequences
X = pad_sequences(X,maxlen=300)

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42, stratify=y)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42, stratify=y_train)


print (X_train.shape,X_test.shape, X_val.shape)

(20000, 300) (25000, 300) (5000, 300)


In [10]:
from keras.layers import *
from keras.models import Model
from keras.callbacks import ModelCheckpoint


input_layer = Input(shape=(300,))
emb_layer = Embedding(5000,100)(input_layer)
conv_layer = Conv1D(100,3, activation="relu")(emb_layer)
pool_layer = GlobalMaxPooling1D()(conv_layer)
hidden_dense_layer = Dense(50,activation="sigmoid")(pool_layer)
dense_layer = Dense(1,activation="sigmoid")(hidden_dense_layer)

model = Model(inputs=input_layer, outputs=dense_layer)
model.compile(loss='binary_crossentropy', optimizer="adam",metrics=["accuracy"])
print (model.summary())
mc = ModelCheckpoint("best_checkpoint.h5", monitor="val_acc" , save_best_only=True, save_weights_only=True)

model.fit(X_train,y_train,validation_data=(X_val,y_val), epochs= 20, batch_size=50, callbacks=[mc])






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 300)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 300, 100)          500000    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 298, 100)          30100     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 50)                5050      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 51        
Total params: 535,201
Trai

In [11]:
model.load_weights("best_checkpoint.h5")
y_test_pred = model.predict(X_test)
y_test_pred = [1 if pred > 0.5 else 0 for pred in y_test_pred ]
import numpy as np
print("Acc:",(1 - np.sum(np.abs(y_test_pred- y_test))/len(y_test_pred)))

Acc: 0.8852
